In [7]:
import pandas as pd

df = pd.read_csv('Acme_Synthetic_Data_final.csv')

# calculating  maximum possible sales
def calculate_max_sales(dataframe, growth_target):
    # Calculating potential max sales for each segment based on max trend
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    
    # Sum up the potential max sales to get a total
    total_max_sales = dataframe['Potential_Max_Sales'].sum()
    
    # Compare the maximum achievable growth with the target
    if total_max_sales < growth_target:
        shortfall = growth_target - total_max_sales
        message = f"Maximum Achievable Growth: ${total_max_sales / 1e6:.2f}M\n" \
                  f"Shortfall: ${shortfall / 1e6:.2f}M\n" \
                  f"The target growth of ${growth_target / 1e6:.2f}M is not achievable under current constraints."
    else:
        message = f"Maximum Achievable Growth: ${total_max_sales / 1e6:.2f}M\n" \
                  "The target growth is achievable."
    
    print(message)
    
    # Returning   total and  detailed dataframe with the new column
    return dataframe


growth_target_input = input("Please enter the growth target (in dollars): ")
try:
    growth_target = float(growth_target_input)  # Convert the input to a float
except ValueError:
    print("Invalid input! Please enter a numerical value.")
    exit()

detailed_report = calculate_max_sales(df, growth_target)

print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Potential_Max_Sales', 'Max_Trend', 'Min_Trend', 'Max_Contribution', 'Min_Contribution']].head())

detailed_report.to_csv('detailed_sales_maximization_report.csv', index=False)


Maximum Achievable Growth: $3.31M
The target growth is achievable.
   Portfolio      Geography Category    Brand      Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A    Body Wash   6625   
1  Hair/Body  North America     Hair  Brand A  Conditioner   3418   
2  Hair/Body  North America     Hair  Brand B         Soap   6639   
3  Hair/Body  North America     Hair  Brand B       Lotion   6459   
4  Hair/Body  North America     Hair  Brand C        Cream   6908   

   Potential_Max_Sales  Max_Trend  Min_Trend  Max_Contribution  \
0          7411.188758   0.118670  -0.043626          0.424121   
1          3950.516148   0.155798  -0.070864          0.455779   
2          8131.637961   0.224829  -0.092702          0.476881   
3          7401.182741   0.145871  -0.114622          0.374270   
4          8551.855388   0.237964  -0.073482          0.342759   

   Min_Contribution  
0          0.096686  
1          0.097888  
2          0.065605  
3          0.083256  
4          

In [10]:
def calculate_max_margin(dataframe, margin_target):
    # Calculating potential max sales for each segment based on max trend
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    
    dataframe['Potential_Max_Margin'] = dataframe['Potential_Max_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Sum up the potential max margin to get a total
    total_max_margin = dataframe['Potential_Max_Margin'].sum()
    
    # Comparing  the maximum achievable margin with the target
    if total_max_margin < margin_target:
        shortfall = margin_target - total_max_margin
        message = f"Maximum Achievable Margin: ${total_max_margin / 1e6:.2f}M\n" \
                  f"Shortfall: ${shortfall / 1e6:.2f}M\n" \
                  f"The target margin of ${margin_target / 1e6:.2f}M is not achievable under current constraints."
    else:
        message = f"Maximum Achievable Margin: ${total_max_margin / 1e6:.2f}M\n" \
                  "The target margin is achievable."
    
    print(message)
    
    return dataframe

margin_target_input = input("Please enter the margin target (in dollars): ")
try:
    margin_target = float(margin_target_input)  # Convert the input to a float
except ValueError:
    print("Invalid input! Please enter a numerical value.")
    exit()

detailed_report = calculate_max_margin(df, margin_target)

print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Potential_Max_Sales', 'Potential_Max_Margin', 'Margin', 'Max_Trend', 'Max_Contribution']].head())

detailed_report.to_csv('Detailed_Sales_With_Margin_report.csv', index=False)

Maximum Achievable Margin: $0.40M
Shortfall: $4.60M
The target margin of $5.00M is not achievable under current constraints.
   Portfolio      Geography Category    Brand      Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A    Body Wash   6625   
1  Hair/Body  North America     Hair  Brand A  Conditioner   3418   
2  Hair/Body  North America     Hair  Brand B         Soap   6639   
3  Hair/Body  North America     Hair  Brand B       Lotion   6459   
4  Hair/Body  North America     Hair  Brand C        Cream   6908   

   Potential_Max_Sales  Potential_Max_Margin  Margin  Max_Trend  \
0          7411.188758           1194.430477      38   0.118670   
1          3950.516148            630.196698      35   0.155798   
2          8131.637961           2249.139501      58   0.224829   
3          7401.182741            941.813720      34   0.145871   
4          8551.855388           1436.301272      49   0.237964   

   Max_Contribution  
0          0.424121  
1          0.

In [11]:
def hit_sales_target_maximize_margin(dataframe, sales_target):
    # Assuming that we can adjust sales up to Max_Trend and use Max_Contribution for margin calculations
    dataframe['Adjusted_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    dataframe['Potential_Margin'] = dataframe['Adjusted_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Initialize total sales and margin
    total_sales = dataframe['Adjusted_Sales'].sum()
    total_margin = dataframe['Potential_Margin'].sum()
    
    # If the total adjusted sales is less than the target, adjust by minimal necessary increments
    if total_sales < sales_target:
        needed_increase = sales_target - total_sales
        dataframe['Additional_Sales'] = (needed_increase / len(dataframe)) * dataframe['Max_Trend']
        dataframe['Adjusted_Sales'] += dataframe['Additional_Sales']
        dataframe['Potential_Margin'] = dataframe['Adjusted_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Recalculate totals
    total_sales = dataframe['Adjusted_Sales'].sum()
    total_margin = dataframe['Potential_Margin'].sum()
    
    return total_sales, total_margin, dataframe

sales_target = 50000000  # $50M

total_sales, total_margin, detailed_report = hit_sales_target_maximize_margin(df, sales_target)

print(f"Total Adjusted Sales to Meet Target: ${total_sales:.2f}")
print(f"Total Potential Margin from Adjusted Sales: ${total_margin:.2f}")

print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Adjusted_Sales', 'Potential_Margin', 'Max_Trend', 'Max_Contribution']].head())

detailed_report.to_csv('detailed_Margin_With_Sales_optimization_report.csv', index=False)

Total Adjusted Sales to Meet Target: $12755880.40
Total Potential Margin from Adjusted Sales: $1554564.72
   Portfolio      Geography Category    Brand      Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A    Body Wash   6625   
1  Hair/Body  North America     Hair  Brand A  Conditioner   3418   
2  Hair/Body  North America     Hair  Brand B         Soap   6639   
3  Hair/Body  North America     Hair  Brand B       Lotion   6459   
4  Hair/Body  North America     Hair  Brand C        Cream   6908   

   Adjusted_Sales  Potential_Margin  Max_Trend  Max_Contribution  
0    18491.577198       2980.210612   0.118670          0.424121  
1    18497.560479       2950.779366   0.155798          0.455779  
2    29124.219704       8055.502873   0.224829          0.476881  
3    21021.395722       2675.010143   0.145871          0.374270  
4    30770.896860       5168.033871   0.237964          0.342759  


In [13]:
import pandas as pd

def hit_margin_target_maximize_sales(dataframe, margin_target):
    # First, calculate potential max sales and margins at their maximum allowed trend and contribution
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    dataframe['Potential_Margin_Amount'] = dataframe['Potential_Max_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Calculate initial total potential margin
    total_potential_margin = dataframe['Potential_Margin_Amount'].sum()
    
    # If the total potential margin is less than the target, adjust by increasing sales within max constraints
    if total_potential_margin < margin_target:
        # Increase sales while trying to maintain or increase margin percentage
        for index, row in dataframe.iterrows():
            if row['Potential_Margin_Amount'] < (row['Margin'] / 100) * margin_target:
                # Increase sales to attempt to meet part of the margin shortfall
                additional_sales_needed = (margin_target - total_potential_margin) / len(dataframe)
                adjusted_sales = min(row['Potential_Max_Sales'] + additional_sales_needed, row['Sales'] * (1 + row['Max_Trend']))
                dataframe.at[index, 'Adjusted_Sales'] = adjusted_sales
                dataframe.at[index, 'Potential_Margin_Amount'] = adjusted_sales * (row['Margin'] / 100) * row['Max_Contribution']
        
        # Recalculate total potential margin
        total_potential_margin = dataframe['Potential_Margin_Amount'].sum()

    return total_potential_margin, dataframe

margin_target = float(input("Please enter the margin target (in dollars): "))

total_potential_margin, detailed_report = hit_margin_target_maximize_sales(df, margin_target)

print(f"Total Potential Margin from Adjusted Sales: ${total_potential_margin:.2f}")

print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Adjusted_Sales', 'Potential_Margin_Amount', 'Max_Trend', 'Max_Contribution']].head())

detailed_report.to_csv('detailed_margin_sales_optimization_report.csv', index=False)

Total Potential Margin from Adjusted Sales: $402240.17
   Portfolio      Geography Category    Brand      Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A    Body Wash   6625   
1  Hair/Body  North America     Hair  Brand A  Conditioner   3418   
2  Hair/Body  North America     Hair  Brand B         Soap   6639   
3  Hair/Body  North America     Hair  Brand B       Lotion   6459   
4  Hair/Body  North America     Hair  Brand C        Cream   6908   

   Adjusted_Sales  Potential_Margin_Amount  Max_Trend  Max_Contribution  
0     7411.188758              1194.430477   0.118670          0.424121  
1     3950.516148               630.196698   0.155798          0.455779  
2     8131.637961              2249.139501   0.224829          0.476881  
3     7401.182741               941.813720   0.145871          0.374270  
4     8551.855388              1436.301272   0.237964          0.342759  


In [14]:
import numpy as np
def project_annual_data(dataframe):
    projections = {}
    
    for year in range(1, 6):  # For each of the next 5 years
        if year == 1:
            # For the first year, use existing data and max trend to project next year's figures
            dataframe[f'Year_{year}_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
            dataframe[f'Year_{year}_Margin'] = dataframe[f'Year_{year}_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
        else:
            growth_trend = np.random.uniform(dataframe['Min_Trend'], dataframe['Max_Trend'])  
            contribution_adjustment = np.random.uniform(dataframe['Min_Contribution'], dataframe['Max_Contribution'])  
            
            dataframe[f'Year_{year}_Sales'] = dataframe[f'Year_{year - 1}_Sales'] * (1 + growth_trend)
            dataframe[f'Year_{year}_Margin'] = dataframe[f'Year_{year}_Sales'] * (dataframe['Margin'] / 100) * contribution_adjustment
        
        projections[f'Year_{year}'] = dataframe[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', f'Year_{year}_Sales', f'Year_{year}_Margin']]
    
    return projections

annual_projections = project_annual_data(df)
for year in annual_projections:
    print(f"Data for {year}:")
    print(annual_projections[year].head(), "\n")  # Displaying only the first few rows for brevity
    annual_projections[year].to_csv(f'{year}_sales_margin_projection.csv', index=False)


Data for Year_1:
   Portfolio      Geography Category    Brand      Segment  Year_1_Sales  \
0  Hair/Body  North America     Hair  Brand A    Body Wash   7411.188758   
1  Hair/Body  North America     Hair  Brand A  Conditioner   3950.516148   
2  Hair/Body  North America     Hair  Brand B         Soap   8131.637961   
3  Hair/Body  North America     Hair  Brand B       Lotion   7401.182741   
4  Hair/Body  North America     Hair  Brand C        Cream   8551.855388   

   Year_1_Margin  
0    1194.430477  
1     630.196698  
2    2249.139501  
3     941.813720  
4    1436.301272   

Data for Year_2:
   Portfolio      Geography Category    Brand      Segment  Year_2_Sales  \
0  Hair/Body  North America     Hair  Brand A    Body Wash   7536.649641   
1  Hair/Body  North America     Hair  Brand A  Conditioner   3931.872866   
2  Hair/Body  North America     Hair  Brand B         Soap   9521.687071   
3  Hair/Body  North America     Hair  Brand B       Lotion   6737.067281   
4  Hair/Body 